# Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (in sklearn, obviously). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the next exercise, you will implement your own LASSO solver, using coordinate descent. 

## The usual

In [1]:
import sklearn
import pandas
import numpy as np

## Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located. *I am so surprised.*

In [2]:
full_data = pandas.read_csv("kc_house_data.csv", index_col=0)

## Create new features

As in Lab 2 (*lab-2.ipynb*), we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
full_data['sqft_living_sqrt'] = full_data['sqft_living'].map(sqrt)
full_data['sqft_lot_sqrt'] = full_data['sqft_lot'].map(sqrt)
full_data['bedrooms_square'] = full_data['bedrooms'] ** 2

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
full_data['floors'] = full_data['floors'].astype(float) 
full_data['floors_square'] = full_data['floors'] ** 2

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [27]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`alpha=l1_penalty`) to the sklearn model `Lasso`. (Other tools may have separate implementations of LASSO). Much like L2/Ridge Regression, the features should be scaled to ensure equal attention inbetween.

In [28]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
l1_penalty=5e4
full_features = scaler.fit_transform(full_data[all_features].values)
full_labels = full_data['price'].values
model_full = Lasso(alpha=l1_penalty).fit(full_features, full_labels)

Find what features had non-zero weight.

In [32]:
# Do you know that even numpy has built-in boolean selector?
coefficients = model_full.coef_
print(coefficients)
print(coefficients[model_full.coef_.nonzero()])

[     0.              0.              0.         132571.09360631
      0.             -0.             -0.              0.
      0.          14623.33961421  29004.06421249      0.
  90207.54789031      0.              0.         -10722.34912003
      0.        ]
[132571.09360631  14623.33961421  29004.06421249  90207.54789031
 -10722.34912003]


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [7]:
from sklearn.model_selection import train_test_split
train_and_validation, test_set = train_test_split(full_data, test_size=0.1, random_state=1)
train_set, validation_set = train_test_split(train_and_validation, test_size=0.5, random_state=1)

In [8]:
train_features = scaler.transform(train_set[all_features].values)
train_labels= train_set['price'].values
valid_features = scaler.transform(validation_set[all_features].values)
valid_labels = validation_set['price'].values

# Next, we write a loop that does the following:
* For `l1_penalty` in 21 steps range between [1, 10^9] (use `np.logspace(0, 9, num=21)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `alpha=l1_penalty` in the parameter.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

In [33]:
l1_penalty_values = np.logspace(0, 9, num=21)
min_error = None
best_l1_penalty = None
validation_errors = []
for l1_penalty in l1_penalty_values:
    model = Lasso(alpha=l1_penalty).fit(train_features, train_labels)
    price_predicted = model.predict(valid_features)
    RSS = ((price_predicted-valid_labels)**2).sum()
    validation_errors.append(RSS)
    if min_error is None or RSS < min_error:
        min_error = RSS
        best_l1_penalty = l1_penalty

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14654475716861.0, tolerance: 127509489470.13391
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14663181882447.375, tolerance: 127509489470.13391
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14687653596508.062, tolerance: 127509489470.13391
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality g

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [34]:
print(validation_errors)
print(best_l1_penalty)
print(min_error)

[416604893858220.7, 416599305725099.06, 416583670464101.6, 416540511736147.9, 416426081845074.7, 416160829225389.4, 415868035478379.56, 416826257573209.1, 434567910677354.8, 462926682778278.9, 509944971743515.25, 685404850598010.8, 1269269269066234.5, 1290758247403054.5, 1290758247403054.5, 1290758247403054.5, 1290758247403054.5, 1290758247403054.5, 1290758247403054.5, 1290758247403054.5, 1290758247403054.5]
501.18723362727246
415868035478379.56


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [35]:
model_best_L1 = Lasso(alpha=best_l1_penalty).fit(train_features, train_labels)
model_best_L1.coef_

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 16474937844501.25, tolerance: 127509489470.13391
  positive)


array([ -14631.87620799,    4773.51870901,   41675.78855477,
        436207.06682844, -410670.42743841,   20538.99499836,
        -38906.99479505,  -16211.26503473,   28604.33669179,
         39232.71729436,   28771.88041433,   17793.15256962,
        151123.85682761,   96018.62012972,   54469.89931089,
        -93946.55668381,    5653.56858066])

In [37]:
np.count_nonzero(model_best_L1.coef_)

17

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 5 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [12]:
max_nonzeros = 5

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [13]:
l1_penalty_values = np.logspace(3, 5, num=21)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(3, 5, num=21)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `alpha=l1_penalty` in the parameter.
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coef_` gives you the coefficients/parameters you learned (barring intercept) in the form of numpy array. You can then use array\[condition\] for the list of values passing the condition. Or just use the builtin `np.count_nonzero()`

In [45]:
min_error = None
best_l1_penalty = None
validation_errors = []
for l1_penalty in l1_penalty_values:
    print (l1_penalty)
    model = Lasso(alpha=l1_penalty).fit(train_features, train_labels)
    print ("# Non zeros : " + str(np.count_nonzero(model.coef_)))
    
    price_predicted = model.predict(valid_features)
    RSS = ((price_predicted-valid_labels)**2).sum()
    validation_errors.append(RSS)
    if min_error is None or RSS < min_error:
        min_error = RSS
        best_l1_penalty = l1_penalty

print ("Best l1_penalty: " + str(best_l1_penalty))
print ("Best RSS: " + str('{0:f}'.format(min_error)))

1000.0
# Non zeros : 15
1258.9254117941675
# Non zeros : 15
1584.893192461114
# Non zeros : 15
1995.2623149688789
# Non zeros : 15
2511.88643150958
# Non zeros : 15
3162.2776601683795
# Non zeros : 14
3981.0717055349733
# Non zeros : 13
5011.872336272725
# Non zeros : 13
6309.57344480193
# Non zeros : 10
7943.282347242814
# Non zeros : 10
10000.0
# Non zeros : 10
12589.254117941662
# Non zeros : 10
15848.93192461114
# Non zeros : 6
19952.62314968879
# Non zeros : 6
25118.864315095823
# Non zeros : 5
31622.776601683792
# Non zeros : 5
39810.71705534969
# Non zeros : 5
50118.72336272725
# Non zeros : 5
63095.7344480193
# Non zeros : 4
79432.82347242821
# Non zeros : 3
100000.0
# Non zeros : 2
Best l1_penalty: 1000.0
Best RSS: 416113886400657.500000


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The smallest `l1_penalty` that has non-zeros equal `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The biggest `l1_penalty` that has non-zeros equal `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [47]:
l1_penalty_min = 19952.62314968879
l1_penalty_max = 50118.72336272725

In [46]:
round(50118.72336272725,0)

50119.0

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [16]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `alpha=l1_penalty`.
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [48]:
min_error = None
best_l1_penalty = None
for l1_penalty in l1_penalty_values:
    model = Lasso(alpha=l1_penalty).fit(train_features, train_labels)
    price_predicted = model.predict(valid_features)
    RSS = ((price_predicted-valid_labels)**2).sum()
    if (min_error is None or RSS < min_error) and (np.count_nonzero(model.coef_) == max_nonzeros):
        min_error = RSS
        best_l1_penalty = l1_penalty

In [49]:
print(best_l1_penalty)

25118.864315095823


In [50]:
round(best_l1_penalty,0)

25119.0

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [51]:
model_max_nonzeros = Lasso(alpha=best_l1_penalty).fit(train_features, train_labels)
model_max_nonzeros.coef_

array([    -0.        ,      0.        ,      0.        , 143610.86073049,
            0.        ,     -0.        ,     -0.        ,      0.        ,
            0.        ,  26127.21193655,  30525.04632111,      0.        ,
       121459.14456365,      0.        ,      0.        , -53061.33575139,
            0.        ])